In [1]:
lines = [line.strip() for line in open('input04.txt').readlines()]

In [2]:
lines[:10]

['[1518-04-25 00:24] falls asleep',
 '[1518-05-22 23:50] Guard #3299 begins shift',
 '[1518-06-25 00:25] falls asleep',
 '[1518-04-13 00:54] wakes up',
 '[1518-04-10 00:04] Guard #2467 begins shift',
 '[1518-08-03 00:01] Guard #2543 begins shift',
 '[1518-03-31 00:01] Guard #613 begins shift',
 '[1518-07-07 00:57] wakes up',
 '[1518-03-28 00:59] wakes up',
 '[1518-07-15 23:59] Guard #883 begins shift']

In [59]:
parsed = []
for line in lines:
    full_date = line[1:17]
    full_date = full_date.replace(':', '-').replace(' ', '-')
    yy = int(line[1:5])
    mm = int(line[6:8])
    dd = int(line[9:11])
    hour = int(line[12:14])
    minute = int(line[15:17])
    if 'falls asleep' in line:
        asleep = True
    else: 
        asleep = False
    if '#' in line:
        guard = int(line.split('#')[1].split(' ')[0])
    else:
        guard = -1
    if 'wakes up' in line:
        wakeup = True
    else:
        wakeup = False
    
    parsed.append((full_date, yy, mm, dd, hour, minute, guard, asleep, wakeup))

In [60]:
parsed[1]

('1518-05-22-23-50', 1518, 5, 22, 23, 50, 3299, False, False)

In [61]:
import pandas as pd

In [62]:
df = pd.DataFrame(parsed, columns=['full_date', 'year', 'month', 'day', 'hour', 'minute', 'guard_id', 'asleep', 'wakeup'])

In [63]:
df.head()

,full_date,year,month,day,hour,minute,guard_id,asleep,wakeup
0,1518-04-25-00-24,1518,4,25,0,24,-1,True,False
1,1518-05-22-23-50,1518,5,22,23,50,3299,False,False
2,1518-06-25-00-25,1518,6,25,0,25,-1,True,False
3,1518-04-13-00-54,1518,4,13,0,54,-1,False,True
4,1518-04-10-00-04,1518,4,10,0,4,2467,False,False


In [69]:
df.describe()

,year,month,day,hour,minute,guard_id
count,1072.0,1072.000000,1072.000000,1072.000000,1072.000000,1072.000000
mean,1518.0,6.767724,15.762127,2.810634,33.986007,421.018657
std,0.0,2.570965,8.832942,7.536439,20.210812,856.590941
min,1518.0,2.000000,1.000000,0.000000,0.000000,-1.000000
25%,1518.0,5.000000,8.000000,0.000000,17.000000,-1.000000
50%,1518.0,7.000000,16.000000,0.000000,39.000000,-1.000000
75%,1518.0,9.000000,24.000000,0.000000,52.000000,283.000000
max,1518.0,11.000000,31.000000,23.000000,59.000000,3571.000000


In [64]:
sorted_data = sorted(parsed, key=lambda item: item[0])
sorted_data[:10]

[('1518-02-22-23-58', 1518, 2, 22, 23, 58, 2543, False, False),
 ('1518-02-23-00-15', 1518, 2, 23, 0, 15, -1, True, False),
 ('1518-02-23-00-41', 1518, 2, 23, 0, 41, -1, False, True),
 ('1518-02-23-23-58', 1518, 2, 23, 23, 58, 3571, False, False),
 ('1518-02-24-00-20', 1518, 2, 24, 0, 20, -1, True, False),
 ('1518-02-24-00-59', 1518, 2, 24, 0, 59, -1, False, True),
 ('1518-02-25-00-01', 1518, 2, 25, 0, 1, 1697, False, False),
 ('1518-02-25-00-08', 1518, 2, 25, 0, 8, -1, True, False),
 ('1518-02-25-00-35', 1518, 2, 25, 0, 35, -1, False, True),
 ('1518-02-26-00-01', 1518, 2, 26, 0, 1, 2161, False, False)]

Let's split the data in shifts.

In [83]:
keys = ['full_date', 'year', 'month', 'day', 'hour', 'minute', 'guard_id', 'asleep', 'wakeup']

In [86]:
shifts = []
shift = []
for line in sorted_data:
    line_dict = dict(zip(keys, line))
    if line_dict['guard_id'] == -1:
        shift.append(line)
    else:
        shifts.append(shift)
        shift = [line]
shifts = shifts[1:]

shifts[0]

[('1518-02-22-23-58', 1518, 2, 22, 23, 58, 2543, False, False),
 ('1518-02-23-00-15', 1518, 2, 23, 0, 15, -1, True, False),
 ('1518-02-23-00-41', 1518, 2, 23, 0, 41, -1, False, True)]

In [110]:
time_vectors = []
for shift in shifts:
    guard_id = dict(zip(keys, shift[0]))['guard_id']
    date = dict(zip(keys, shift[0]))['full_date']
    time_vector = [0 for _ in range(60)]
    for start, end in zip(shift[1::2], shift[2::2]):
        start_min = dict(zip(keys, start))['minute']
        end_min = dict(zip(keys, end))['minute']
        time_vector[start_min:end_min] = [1 for _ in range(start_min, end_min)]
    time_vectors.append((full_date, guard_id, time_vector))

In [113]:
guard_ids = set(item[1] for item in time_vectors)
guard_ids

{157,
 263,
 283,
 449,
 577,
 613,
 883,
 919,
 953,
 1091,
 1307,
 1571,
 1597,
 1607,
 1619,
 1697,
 2161,
 2423,
 2467,
 2543,
 2801,
 3299,
 3571}

Who sleeps the most?

In [117]:
sleepers = {}
for this_guard_id in guard_ids:
    slept = 0
    for full_date, guard_id, time_vector in time_vectors:
        if guard_id == this_guard_id:
            slept += sum(time_vector)
    sleepers[this_guard_id] = slept

sleepers

{157: 0,
 263: 0,
 283: 245,
 449: 450,
 577: 0,
 613: 325,
 883: 238,
 919: 290,
 953: 384,
 1091: 185,
 1307: 218,
 1571: 493,
 1597: 312,
 1607: 140,
 1619: 454,
 1697: 337,
 2161: 459,
 2423: 251,
 2467: 259,
 2543: 150,
 2801: 408,
 3299: 390,
 3571: 263}

In [120]:
sorted(sleepers, key=lambda item: sleepers[item], reverse=True)

[1571,
 2161,
 1619,
 449,
 2801,
 3299,
 953,
 1697,
 613,
 1597,
 919,
 3571,
 2467,
 2423,
 283,
 883,
 1307,
 1091,
 2543,
 1607,
 263,
 157,
 577]

Let's now look at guard 1571.

In [121]:
import numpy as np

In [124]:
average = np.zeros((60,))
for full_date, guard_id, time_vector in time_vectors:
    if guard_id == 1571:
        average += time_vector
average

array([  1.,   2.,   2.,   2.,   2.,   3.,   5.,   5.,   5.,   6.,   9.,
         9.,   9.,   9.,   9.,  10.,  10.,  10.,   9.,   9.,   9.,  10.,
        11.,  11.,  10.,  10.,   9.,  11.,  11.,   9.,   9.,   9.,   9.,
        10.,  10.,  10.,  10.,  10.,  10.,   7.,   7.,   7.,   7.,   8.,
         9.,  10.,  10.,  11.,  11.,  10.,  10.,   9.,  10.,  11.,  12.,
         9.,  11.,   7.,   3.,   0.])

In [125]:
average.max()

12.0

In [127]:
(average==12).nonzero()

(array([54]),)

In [128]:
1571 * 54

84834

## part 2 

In [140]:
averages = {}
for this_guard_id in guard_ids:
    average = np.zeros((60,))
    for full_date, guard_id, time_vector in time_vectors:
        if guard_id == this_guard_id:
            average += time_vector
    max_minutes = max(average)
    minute = (average==max_minutes).nonzero()[0]
    print("guard {}, max_minutes_slept {}, when {}".format(this_guard_id, max_minutes, minute))

guard 263, max_minutes_slept 0.0, when [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49
 50 51 52 53 54 55 56 57 58 59]
guard 919, max_minutes_slept 8.0, when [43 46 47 49 50 51]
guard 283, max_minutes_slept 9.0, when [31 33]
guard 1307, max_minutes_slept 7.0, when [29 30]
guard 157, max_minutes_slept 0.0, when [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49
 50 51 52 53 54 55 56 57 58 59]
guard 1697, max_minutes_slept 11.0, when [34 35 36 37 38]
guard 1571, max_minutes_slept 12.0, when [54]
guard 2467, max_minutes_slept 9.0, when [37]
guard 953, max_minutes_slept 11.0, when [21 24 25 26]
guard 1597, max_minutes_slept 9.0, when [27 35 36 37 44 45 46]
guard 449, max_minutes_slept 12.0, when [43]
guard 577, max_minutes_slept 0.0, when [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 1

In [136]:
33 * 1619

53427